<a href="https://colab.research.google.com/github/pratap977/CS6910/blob/main/Assignment-1/Question%203%2C4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb
from keras.datasets import fashion_mnist
import numpy as np
from  matplotlib import pyplot as plt
import time
import math
import wandb
wandb.login()





     |████████████████████████████████| 1.7 MB 4.9 MB/s 
     |████████████████████████████████| 181 kB 51.9 MB/s 
     |████████████████████████████████| 144 kB 48.8 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=0486eb5f5e5c3ad976356c849e7df70208ec6119673aa23557f8da85585eb0ca
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
((xtrain, ytrain), (xtest, ytest)) = fashion_mnist.load_data()

items=["T-shirt/top", "Trouser","Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag","Ankle boot"]
ftrx=np.zeros((60000,784))
ftry=np.zeros((60000,10))
ftex=np.zeros((10000,784))
ftey=np.zeros((10000,10))
for i in range(len(xtrain)):
  ftrx[i]=xtrain[i].flatten();
  ftrx[i]=ftrx[i]/256
  ftry[i][ytrain[i]]=1
for i in range(len(xtest)):
  ftex[i]=xtest[i].flatten();
  ftex[i]=ftex[i]/256
  ftey[i][ytest[i]]=1




     # activation functions and gradients

def sigmoid(x):
  return np.where(x>=0, 1/(1+np.exp(-x)), np.exp(x)/(1+np.exp(x)))

def grad_sigmoid(z):
  return z*(1-z)

def softmax(z):
  mx=np.max(z)
  z=z-mx
  out=(np.exp(z)+1e-9)/np.sum(np.exp(z)+1e-9)
  return out

def grad_softmax(yh,ty):
  return yh-ty

def Relu(x):
    return np.maximum(0, x)

def grad_Relu(x):
    return [1 if xi>0 else 0 for xi in x]

def Tanh(x):
  a= 2*np.where(x>=0, 1/(1+np.exp(-2*x)), np.exp(2*x)/(1+np.exp(2*x)))
  return a-1


def grad_Tanh(x):
    return 1-Tanh(x)**2

def InitParams(x,init):
    weights=[]
    baises=[]
    if(init=='random'):
      weights.append((np.random.randn(x[0],784)))
      baises.append((np.random.randn(x[0])))
      l=len(x)-1
      for i in range(l):
          weights.append(0.1*(np.random.randn(x[i+1],x[i])))
          baises.append(0.1*(np.random.randn(x[i+1])))
      weights.append(0.1*(np.random.randn(10,x[-1])))
      baises.append(0.1*(np.random.randn(10)))
    else:
      weights.append(np.random.randn(x[0], 784)*np.sqrt(2./(784)))
      baises.append(np.zeros(x[0]))
      l=len(x)-1
      for i in range(l):
          weights.append(np.random.randn(x[i+1], x[i])*np.sqrt(2./(x[i])))
          baises.append(np.zeros(x[i+1]))
      weights.append(np.sqrt(2./(x[-1]))*(np.random.randn(10,x[-1])))
      baises.append((np.zeros(10)))


    return weights,baises



def initgrads(N,Nunits):
  dh,da,dw,db=[],[],[],[]
  dw.append(np.zeros((Nunits[0],784)))
  db.append(np.zeros(Nunits[0]))
  da.append(np.zeros(Nunits[0]))
  dh.append(np.zeros(Nunits[0]))
  for i in range(1,N):
    dw.append(np.zeros((Nunits[i],Nunits[i-1])))
    db.append(np.zeros(Nunits[i]))
    da.append(np.zeros(Nunits[i]))
    dh.append(np.zeros(Nunits[i]))
  dw.append(np.zeros((10,Nunits[-1])))
  db.append(np.zeros(10))
  da.append(np.zeros(10))
  dh.append(np.zeros(10))
  return dw,db,da,dh

  
def Feed_Frwd_Nw1(xTr,Weighs,Bais,activation,loss):
  a_i=[]
  h_i=[]
  a_i.append(np.dot(Weighs[0],xTr)+Bais[0])       
  if (activation=='sig'):
    h_i.append(sigmoid(a_i[0]))
  elif (activation=='tanh'):
    h_i.append(Tanh(a_i[0]))
  elif (activation=='relu'):
    h_i.append(Relu(a_i[0]))
  for i in range(1,len(Weighs)-1):
    a_i.append((np.dot(Weighs[i],h_i[i-1])+Bais[i]))
    if (activation=='sig'):
      h_i.append(sigmoid(a_i[i]))
    elif (activation=='tanh'):
      h_i.append(Tanh(a_i[i]))
    elif (activation=='relu'):
      h_i.append(Relu(a_i[i]))
  a_i.append(np.dot(Weighs[-1],h_i[-1])+Bais[-1])
  h_i.append(softmax(a_i[-1])) 
  yp=h_i[-1]
  return yp,a_i,h_i


def Back_Prop(ip,ypr,ty,a_i,h_i,W,B,N,nrl,activation,Loss):
  k=len(ypr)
  dw,db,da,dh=initgrads(N,nrl)
  if (Loss=="ce"):
    dh[-1]=[-(t/ypr) if t==1 else 0 for t in ty ]
    da[-1]=ypr-ty
  elif (Loss=="mse"):
    dh[-1]=(ypr-ty)
    da[-1]=dh[-1]*(ypr-ypr**2)

  db[-1]=da[-1] 
  dw[-1]=np.dot((da[-1][:,np.newaxis]),(h_i[-2][:,np.newaxis]).T)
 
  for i in range(N-1,-1,-1):
    dh[i]=np.squeeze(np.dot(W[i+1].T,da[i+1]))

    if (activation=='sig'):
      da[i]=dh[i]*grad_sigmoid(h_i[i])
    elif (activation=='tanh'):
      da[i]=dh[i]*grad_Tanh(a_i[i])
    elif (activation=='relu'):
      da[i]=dh[i]*grad_Relu(a_i[i])
    db[i]=np.copy(da[i])
    if (i==0):
      dw[i]=np.dot(da[i][:,np.newaxis],ip[:,np.newaxis].T)
    else:
      dw[i]=np.dot(da[i][:,np.newaxis],h_i[i-1][:,np.newaxis].T)
  return dw,db


def sum_weights(W):
  sum=0
  for i in range(len(W)):
    sum+=np.sum(W[i])
  return sum



           # Losses and accuracies
def val_acc_loss(W,B,activation,ls_fun,wd):
  cost=0
  count=0
  if (ls_fun=='ce'):
    for i in range(len(ftrx[54000:])):
      ypr,_,_=Feed_Frwd_Nw1(ftrx[i],W,B,activation,ls_fun)
      cost += (np.sum(np.multiply(ftry[i],np.log(ypr))) + wd/2*sum_weights(W) )
      if (np.argmax(ypr) ==np.argmax(ftry[i])):
        count+=1
    return -cost/6000, count/6000
    #return np.sum(ay.multiply(np.log(yh)))
  elif (ls_fun=='mse'):
    for i in range(len(ftrx[54000:])):
      ypr,_,_=Feed_Frwd_Nw1(ftrx[i],W,B,activation,ls_fun)
      cost+=(0.5*np.sum((ftry[i]- ypr)**2) + wd/2*sum_weights(W) )
      if (np.argmax(ypr) ==np.argmax(ftry[i])):
        count+=1
    return cost/6000, count/6000

def train_acc_loss(W,B,activation,ls_fun,wd):
  cost=0
  count=0
  if (ls_fun=='ce'):
    for i in range(len(ftrx[:54000])):
      ypr,_,_=Feed_Frwd_Nw1(ftrx[i],W,B,activation,ls_fun)
      cost += (np.sum(np.multiply(ftry[i],np.log(ypr))) + wd/2*sum_weights(W) )
      if (np.argmax(ypr) ==np.argmax(ftry[i])):
        count+=1
    return -cost/54000, count/54000
    #return np.sum(ay.multiply(np.log(yh)))
  elif (ls_fun=='mse'):
    for i in range(len(ftrx[:54000])):
      ypr,_,_=Feed_Frwd_Nw1(ftrx[i],W,B,activation,ls_fun)
      cost+=(0.5*np.sum((ftry[i]- ypr)**2) + wd/2*sum_weights(W) )
      if (np.argmax(ypr) ==np.argmax(ftry[i])):
        count+=1
    return cost/54000, count/54000




def test_acc(W,B,activation,ls_fun):
  count=0
  predictions=[]
  matrix=np.zeros((10,10))
  for i in range(len(ftex)):
    ypr,_,_=Feed_Frwd_Nw1(ftex[i],W,B,activation,ls_fun)
    if (np.argmax(ypr) == ytest[i]):
      count+=1
    predictions.append(np.argmax(ypr))
    matrix[ytest[i]][np.argmax(ypr)]+=1
    
  lables={}
  for i in range(len(items)):
    lables[i]=items[i]
  wandb.log({"confusion matrix": wandb.plot.confusion_matrix(probs=None,
                                                              y_true=ytest,
                                                              preds=predictions,
                                                              class_names=items)})
  wandb.log({"confusion matrix v2.0": wandb.plots.HeatMap(items, items, matrix, show_text=True),"Accuracy":count/len(ytest)})
  return count/len(ytest)



def gradDecent(ftrx,ftry,init,wd,eta,N,nrl,activation,Ls_fun,epochs,batchsize):
  W,B=InitParams(nrl,init)
  e=0
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        for k in range(len(gw)):
          W[k]=np.subtract(W[k],eta*uw[k]/batchsize)
          B[k]=np.subtract(B[k],eta*ub[k]/batchsize)
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun,wd)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun,wd)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun)  
  return hi,W,B


import copy 
def Momentum(ftrx,ftry,init,batchsize,gamma,wd,eta,N,nrl,activation,Ls_fun,epochs=1):
  W,B=InitParams(nrl,init)
  vw,vb,_,_=initgrads(N,nrl)
  pw,pb,_,_=initgrads(N,nrl)
  e=0
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        for k in range(len(gw)):
          # z1,z2=W[k],B[k]
          vw[k]=np.add(gamma*pw[k],eta*uw[k]/batchsize)
          vb[k]=np.add(gamma*pb[k],eta*ub[k]/batchsize)
          W[k]=np.subtract(W[k],vw[k])
          B[k]=np.subtract(B[k],vb[k])
          pw[k]=copy.deepcopy(vw[k])
          pb[k]=copy.deepcopy(vb[k])
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun,wd)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun,wd)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun)  
  return hi,W,B

import copy 
def Nesterov(ftrx,ftry,init,batchsize,gamma,wd,eta,N,nrl,activation,Ls_fun,epochs=1):
  W,B=InitParams(nrl,init)
  vw,vb,_,_=initgrads(N,nrl)
  pw,pb,_,_=initgrads(N,nrl)
  w,b=copy.deepcopy(W),copy.deepcopy(B)
  e=0
  while(epochs>0):
    start = time.time()
    e+=1
    start = time.time()
    epochs-=1 
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx)):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun) 
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,w,b,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        for k in range(len(gw)):
          # z1,z2=W[k],B[k]
          vw[k]=np.add(gamma*pw[k],eta*uw[k]/batchsize)
          vb[k]=np.add(gamma*pb[k],eta*ub[k]/batchsize)
          W[k]=np.subtract(W[k],vw[k])
          B[k]=np.subtract(B[k],vb[k])
          pw[k]=copy.deepcopy(vw[k])
          pb[k]=copy.deepcopy(vb[k])
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k])
        for k in range(len(gw)):
          w[k]=np.subtract(W[k],gamma*pw[k])
          b[k]=np.subtract(B[k],gamma*pb[k])    
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun,wd)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun,wd)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun)  
  return hi,W,B

def rmsprop(ftrx,ftry,init,batchsize,eps,beta,wd,eta,N,nrl,activation,Ls_fun,epochs=1):
  W,B=InitParams(nrl,init)
  vw,vb,_,_=initgrads(N,nrl)
  e=0
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        for k in range(len(gw)):
          vw[k]=np.add(beta*vw[k],(1-beta)*uw[k]**2)
          vb[k]=np.add(beta*vb[k],(1-beta)*ub[k]**2)
          W[k]=np.subtract(W[k],((eta/batchsize)/np.sqrt(vw[k]+eps))*uw[k])
          B[k]=np.subtract(B[k],((eta/batchsize)/np.sqrt(vb[k]+eps))*ub[k])
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun,wd)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun,wd)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun)  
  return hi,W,B




def Adam(ftrx,ftry,init,batchsize,eps,beta1,beta2,wd,eta,N,nrl,activation,Ls_fun,epochs=1):
  W,B=InitParams(nrl,init)
  vw,vb,_,_=initgrads(N,nrl)
  mw,mb,_,_=initgrads(N,nrl)
  beta1=0.9
  e=0
  t=1
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        t+=1
        for k in range(len(gw)):
          vw[k]=np.add(beta2*vw[k],(1-beta2)*uw[k]**2)
          vb[k]=np.add(beta2*vb[k],(1-beta2)*ub[k]**2)

          mw[k]=np.add(beta1*mw[k],(1-beta1)*uw[k])
          mb[k]=np.add(beta1*mb[k],(1-beta1)*ub[k])
          mw[k]/=(1-np.power(beta1,t,dtype=np.float64))
          mb[k]/=(1-np.power(beta1,t,dtype=np.float64))
          vw[k]=np.divide(vw[k],(1-np.power(beta2,t,dtype=np.float64)))
          vb[k]=np.divide(vb[k],(1-np.power(beta2,t,dtype=np.float64)))
          W[k]=np.subtract(W[k],((eta/batchsize)/np.sqrt(vw[k]+eps))*mw[k])
          B[k]=np.subtract(B[k],((eta/batchsize)/np.sqrt(vb[k]+eps))*mb[k])
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun,wd)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun,wd)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun)  
  return hi,W,B








def NAdam(ftrx,ftry,init,batchsize,eps,beta1,beta2,wd,eta,N,nrl,activation,Ls_fun,epochs=1):
  W,B=InitParams(nrl,init)
  vw,vb,_,_=initgrads(N,nrl)
  mw,mb,_,_=initgrads(N,nrl)
  beta1=0.9
  e=0
  t=1
  while(epochs>0):
    e+=1
    start = time.time()
    epochs-=1
    uw,ub,_,_=initgrads(N,nrl)
    for i in range(len(ftrx[:54000])):
      pred_y,ai,hi=Feed_Frwd_Nw1(ftrx[i],W,B,activation,Ls_fun)
      gw,gb=Back_Prop(ftrx[i],pred_y,ftry[i],ai,hi,W,B,N,nrl,activation,Ls_fun)
      for j in range(len(gw)):
        uw[j]=np.add(uw[j],gw[j]+wd*W[j])
        ub[j]=np.add(ub[j],gb[j]+wd*B[j])
      if((i+1)%batchsize==0 or i==53999):
        t+=1
        for k in range(len(gw)):
          vw[k]=np.add(beta2*vw[k],(1-beta2)*uw[k]**2)
          vb[k]=np.add(beta2*vb[k],(1-beta2)*ub[k]**2)

          mw[k]=np.add(beta1*mw[k],(1-beta1)*uw[k])
          mb[k]=np.add(beta1*mb[k],(1-beta1)*ub[k])
          mw[k]/=(1-np.power(beta1,t,dtype=np.float64))
          mb[k]/=(1-np.power(beta1,t,dtype=np.float64))
          vw[k]=np.divide(vw[k],(1-np.power(beta2,t,dtype=np.float64)))
          vb[k]=np.divide(vb[k],(1-np.power(beta2,t,dtype=np.float64)))
          W[k]=np.subtract(W[k],((eta/batchsize)/np.sqrt(vw[k]+eps))*((beta1*mw[k]+(1-beta1)*uw[k])/(1-beta1**t)))
          B[k]=np.subtract(B[k],((eta/batchsize)/np.sqrt(vb[k]+eps))*((beta1*mb[k]+(1-beta1)*ub[k])/(1-beta1**t)))
          uw[k]=np.subtract(uw[k],uw[k])
          ub[k]=np.subtract(ub[k],ub[k]) 
            
    end = time.time() 
    tloss,tacc=train_acc_loss(W,B,activation,Ls_fun,wd)
    vloss,vacc=val_acc_loss(W,B,activation,Ls_fun,wd)
    wandb.log({"epoch":e,"Train_loss":tloss,"Train_acc":tacc,"val_loss":vloss,"val_Accuracy":vacc})
    #print("epochs :",e,"train_loss :",tloss,"  Train_acc:",tacc,"val_loss:",vloss,"val_Accuracy:",vacc,"    time:",math.ceil(end-start))
  test_acc(W,B,activation,Ls_fun)  
  return hi,W,B


4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
lr=[1e-3,1e-4,1e-2,1e-5,2e-3,2e-4,2e-2,2e-5,3e-3,3e-4,3e-2,3e-5,4e-3,4e-4,4e-2,4e-5,5e-3,5e-4,5e-2,5e-5,6e-3,6e-4,6e-2,6e-5,7e-3,7e-4,7e-2,7e-4,8e-3,8e-4,8e-2,8e-5,9e-3,9e-4,9e-2,9e-5,1e-1,2e-1,3e-1,4e-1,5e-1,6e-1,1.5e-3,1.5e-4,1.5e-2,1.5e-5,2.5e-3,2.5e-4,2.5e-2,2.5e-5,3.5e-3,3.5e-4,3.5e-2,3.5e-5,4.5e-3,4.5e-4,4.5e-2,4.5e-5,5.5e-3,5.5e-4,5.5e-2,5.5e-5,6.5e-3,6.5e-4,6.5e-2,6.5e-5,7.5e-3,7.5e-4,7.5e-2,7.5e-4,8.5e-3,8.5e-4,8.5e-2,8.5e-5,9.5e-3,9.5e-4,9.5e-2,9.5e-5,1.5e-1,2.5e-1,3.5e-1,4.5e-1,5e-1,6.5e-1]
len(lr)

In [ ]:
val=[0.065, 0.06, 0.25, 0.35, 0.00045, 0.00055, 5.5e-06, 0.0001, 1.5e-05, 0.045, 0.05, 0.0035, 6.5e-05, 0.04, 0.0007, 0.00075, 0.035, 0.095, 0.1, 0.0008, 0.0003, 0.025, 4.5e-05, 0.00035, 0.02, 0.085, 0.015, 0.075, 0.08, 0.0009, 9.5e-05, 0.0004, 1e-05, 0.01, 0.009, 0.0095, 0.0085, 0.0075, 0.008, 0.00095, 0.0055, 0.006, 0.0045, 0.005, 0.004, 0.007, 0.003, 0.0025, 0.002, 0.0015, 0.0065]
print(val)

[0.065, 0.06, 0.25, 0.35, 0.00045, 0.00055, 5.5e-06, 0.0001, 1.5e-05, 0.045, 0.05, 0.0035, 6.5e-05, 0.04, 0.0007, 0.00075, 0.035, 0.095, 0.1, 0.0008, 0.0003, 0.025, 4.5e-05, 0.00035, 0.02, 0.085, 0.015, 0.075, 0.08, 0.0009, 9.5e-05, 0.0004, 1e-05, 0.01, 0.009, 0.0095, 0.0085, 0.0075, 0.008, 0.00095, 0.0055, 0.006, 0.0045, 0.005, 0.004, 0.007, 0.003, 0.0025, 0.002, 0.0015, 0.0065]


In [4]:
s_config = {
                  'method'    : "random",
                  'metric'    : {'name': 'val_Accuracy','goal':'maximize'},
                  'parameters': {
                                  'optimizer'              : {'values': ['sgd','gd','momentum','nesterov','rmsprop','adam','nadam']},
                                  'hidden_layer_size'        : {'values':[32, 64,16,20,128,50]},
                                  'num_hidden_layers'                  : {'values':[1,2,3,5] },
                                  'activation'         : {'values':['sig','relu','tanh']},
                                  'lossfun'            : {'values':['ce']},
                                  'lr'       : {'values':[0.065, 0.06, 0.25, 0.35, 0.00045, 0.00055, 5.5e-06, 0.0001, 1.5e-05, 0.045, 0.05, 0.0035, 6.5e-05, 0.04, 0.0007, 0.00075, 0.035, 0.095, 0.1, 0.0008, 0.0003, 0.025, 4.5e-05, 0.00035, 0.02, 0.085, 0.015, 0.075, 0.08, 0.0009, 9.5e-05, 0.0004, 1e-05, 0.01, 0.009, 0.0095, 0.0085, 0.0075, 0.008, 0.00095, 0.0055, 0.006, 0.0045, 0.005, 0.004, 0.007, 0.003, 0.0025, 0.002, 0.0015, 0.0065]},
                                   'weights_initializer': {'values':['random','Xavier']},
                                  'epochs'             :{'values':[5,10,6,8,15]},
                                  'weightDecay'        :{'values':[0.005,0,0.00005,0.0005,0.0006]},
                                  'batchsize'         :{'values':[16,32,64,128]},

                                }
                }

In [5]:
def train():

  hyperparameter=dict(
      epochs = 5,                                       
      batchsize = 10,                                                    
      weightDecay = 0.00005,                                            
      hidden_layer_size = 128,
      weights_initializer='Xaviuor',                               
      lr=0.0095,
      activation='tanh',                                             
      optimizer = "adam",                      
      num_hidden_layers= 3,           
      lossfun = "ce",
      
  )

  wandb.init(config=hyperparameter)

  config=wandb.config
  wandb.run.name = "e_{}_hls_{}_numhl_{}_opt_{}_bs_{}_init_{}_ac_{}_loss_{}_learning_rate_{}_wdecay_{}".format(config.epochs,\
                                                                      config.hidden_layer_size,\
                                                                      config.num_hidden_layers,\
                                                                      config.optimizer,\
                                                                      config.batchsize,\
                                                                      config.weights_initializer,\
                                                                      config.activation,\
                                                                      config.lossfun,\
                                                                      config.lr,\
                                                                      config.weightDecay)
  output_size=10                                      
  nrl=[config.hidden_layer_size for i in range(config.num_hidden_layers)]   #hidden layer sizes array creation
  if(config.optimizer=="sgd"):
    param=gradDecent(ftrx,ftry,config.weights_initializer,config.weightDecay,config.lr,config.num_hidden_layers,nrl,config.activation,config.lossfun,config.epochs,1)
  if(config.optimizer=="gd"):
    param=gradDecent(ftrx,ftry,config.weights_initializer,config.weightDecay,config.lr,config.num_hidden_layers,nrl,config.activation,config.lossfun,config.epochs,config.batchsize)
  elif(config.optimizer=="adam"):
    param=Adam(ftrx,ftry,config.weights_initializer,config.batchsize,1e-8,0.9,0.9,config.weightDecay,config.lr,config.num_hidden_layers,nrl,config.activation,config.lossfun,config.epochs)
  elif(config.optimizer=="nadam"):
    param=NAdam(ftrx,ftry,config.weights_initializer,config.batchsize,1e-8,0.9,0.9,config.weightDecay,config.lr,config.num_hidden_layers,nrl,config.activation,config.lossfun,config.epochs)
  elif(config.optimizer=="momentum"):
    param=Momentum(ftrx,ftry,config.weights_initializer,config.batchsize,0.9,config.weightDecay,config.lr,config.num_hidden_layers,nrl,config.activation,config.lossfun,config.epochs)
  elif(config.optimizer=="nesterov"):
    param=Nesterov(ftrx,ftry,config.weights_initializer,config.batchsize,0.9,config.weightDecay,config.lr,config.num_hidden_layers,nrl,config.activation,config.lossfun,config.epochs)
  elif(config.optimizer=="rmsprop"):
    param=rmsprop(ftrx,ftry,config.weights_initializer,config.batchsize,1e-8,0.8,config.weightDecay,config.lr,config.num_hidden_layers,nrl,config.activation,config.lossfun,config.epochs)

In [ ]:
sweep_id = wandb.sweep(s_config,project='CS6910-assignment-1')
wandb.agent(sweep_id,train,project='CS6910-assignment-1',count=150)


Create sweep with ID: 9axhkxy6
Sweep URL: https://wandb.ai/pratap49/CS6910-assignment-1/sweeps/9axhkxy6


wandb: Agent Starting Run: 9h3sxlbp with config:
wandb: 	activation: relu
wandb: 	batchsize: 32
wandb: 	epochs: 8
wandb: 	hidden_layer_size: 16
wandb: 	lossfun: ce
wandb: 	lr: 0.0025
wandb: 	num_hidden_layers: 2
wandb: 	optimizer: sgd
wandb: 	weightDecay: 0.0005
wandb: 	weights_initializer: random
wandb: Currently logged in as: pratap49 (use `wandb login --relogin` to force relogin)


wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: Visualizing heatmap.


Accuracy,▁
Train_acc,▁▃▅▆▇▇██
Train_loss,█▆▄▃▂▂▁▁
epoch,▁▂▃▄▅▆▇█
val_Accuracy,▁▃▅▆▇▇██
val_loss,█▆▄▃▂▂▁▁
Accuracy,0.8121
Train_acc,0.82863
Train_loss,0.4583
epoch,8
val_Accuracy,0.82817


wandb: Agent Starting Run: ab478m0v with config:
wandb: 	activation: sig
wandb: 	batchsize: 16
wandb: 	epochs: 6
wandb: 	hidden_layer_size: 50
wandb: 	lossfun: ce
wandb: 	lr: 0.003
wandb: 	num_hidden_layers: 3
wandb: 	optimizer: momentum
wandb: 	weightDecay: 0.005
wandb: 	weights_initializer: Xavier


...

Accuracy,▁
Train_acc,▁▁▁▁▃▅▆▇▇█
Train_loss,█▄▂▂▂▁▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
val_Accuracy,▁▁▁▁▃▄▆▆▇█
val_loss,█▄▂▂▂▁▁▁▁▁
Accuracy,0.2854
Train_acc,0.28206
Train_loss,2.29883
epoch,10
val_Accuracy,0.28417


wandb: Agent Starting Run: bo27z4cb with config:
wandb: 	activation: tanh
wandb: 	batchsize: 128
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 128
wandb: 	lossfun: ce
wandb: 	lr: 0.0001
wandb: 	num_hidden_layers: 2
wandb: 	optimizer: nesterov
wandb: 	weightDecay: 0.0005
wandb: 	weights_initializer: random


wandb: Visualizing heatmap.


Accuracy,▁
Train_acc,▁▄▅▆▇▇▇███
Train_loss,█▆▄▃▃▂▂▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
val_Accuracy,▁▄▅▆▇▇▇███
val_loss,█▆▄▃▃▂▂▁▁▁
Accuracy,0.6822
Train_acc,0.69169
Train_loss,0.8447
epoch,10
val_Accuracy,0.6965


wandb: Agent Starting Run: kdvboj5u with config:
wandb: 	activation: tanh
wandb: 	batchsize: 64
wandb: 	epochs: 15
wandb: 	hidden_layer_size: 64
wandb: 	lossfun: ce
wandb: 	lr: 0.00035
wandb: 	num_hidden_layers: 5
wandb: 	optimizer: nadam
wandb: 	weightDecay: 0.0006
wandb: 	weights_initializer: Xavier
